In [1]:
import os
import torch
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
from google.colab import drive
import gdown
import json

# ----------------------------------------
# Check GPU
# ----------------------------------------
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ----------------------------------------
# Clone repository (if not already)
# ----------------------------------------
repo_path = "/content/radiology-agent-adk-capstone"
if not os.path.exists(repo_path):
    !git clone https://github.com/reyhan1994/radiology-agent-adk-capstone.git
os.chdir(repo_path)
print("Repo files:", os.listdir(repo_path))

# ----------------------------------------
# Install dependencies
# ----------------------------------------
!pip install -q torch torchvision kaggle gdown

GPU Available: True
GPU Name: Tesla T4
Using device: cuda
Repo files: ['memory', 'sample_images', 'main.py', 'training', 'models', 'requirements.txt', 'utils', '.gitignore', '.git', 'LICENSE', 'README.md', 'master_agent.py', 'agents', 'adk']


In [4]:
# Setup Kaggle API (environment variables recommended)
# ----------------------------------------
os.environ["KAGGLE_USERNAME"] = "your_username"
os.environ["KAGGLE_KEY"] = "your_api_key"
kaggle_username = os.environ.get("KAGGLE_USERNAME")
kaggle_key      = os.environ.get("KAGGLE_KEY")
if kaggle_username and kaggle_key:
    os.makedirs("/root/.kaggle", exist_ok=True)
    with open("/root/.kaggle/kaggle.json", "w") as f:
        json.dump({"username": kaggle_username, "key": kaggle_key}, f)
    os.chmod("/root/.kaggle/kaggle.json", 0o600)
    !kaggle -v
else:
    print("⚠️ Kaggle credentials not set. Set KAGGLE_USERNAME and KAGGLE_KEY env variables.")


Kaggle API 1.7.4.5


In [5]:
# Download dataset
# ----------------------------------------
dataset_dir = "/content/pneumonia_dataset"
if not os.path.exists(dataset_dir):
    !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia --unzip -p {dataset_dir}

# Fix folder structure
train_dir = os.path.join(dataset_dir, "chest_xray/train")
test_dir  = os.path.join(dataset_dir, "chest_xray/test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 98% 2.25G/2.29G [00:19<00:00, 212MB/s]
100% 2.29G/2.29G [00:19<00:00, 126MB/s]


In [6]:
# Mount Google Drive for storing model weights
# ----------------------------------------
drive.mount('/content/drive')
weights_path = "/content/drive/MyDrive/radiology_agent/models/chest_classifier.pt"
os.makedirs(os.path.dirname(weights_path), exist_ok=True)

# ----------------------------------------
# Download fine-tuned weights if not present
# ----------------------------------------
if not os.path.exists(weights_path):
    gdown.download(
        url="https://drive.google.com/uc?id=1mDpUmGjR5OKXodd8DxFJVsR-iMsrPuIb",
        output=weights_path,
        quiet=False
    )
print("✅ Model weights path:", weights_path)

# ----------------------------------------
# Image transformations
# ----------------------------------------
tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# ----------------------------------------
# Create dataset and dataloader (optional, if fine-tuning)
# ----------------------------------------
train_ds = datasets.ImageFolder(train_dir, transform=tf)
test_ds  = datasets.ImageFolder(test_dir, transform=tf)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=16, shuffle=False)

# ----------------------------------------
# Load model (optional, for inference/fine-tuning)
# ----------------------------------------
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 3)
model = model.to(device)
# Load fine-tuned weights
state = torch.load(weights_path, map_location=device)
model.load_state_dict(state)
model.eval()
print("✅ Loaded fine-tuned ResNet18 weights.")



Mounted at /content/drive
✅ Model weights path: /content/drive/MyDrive/radiology_agent/models/chest_classifier.pt
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 159MB/s]


✅ Loaded fine-tuned ResNet18 weights.


In [7]:
# Run pipeline on sample images
# ----------------------------------------
!PYTHONPATH=. python main.py --input sample_images --output submission.csv



Found 10 images
✅ Loaded fine-tuned model weights from /content/drive/MyDrive/radiology_agent/models/chest_classifier.pt
Processing: sample_images/005.jpeg
-> artifacts returned: {'user_request': 'sample_images/005.jpeg', 'patient_data': {'patient_id': 'sample_images/005.jpeg', 'name': 'Ali Ahmadi', 'age': 45}, 'analysis_findings': {'pathology': 'Pneumonia', 'confidence': 0.9997}, 'coding_result': {'ICD_10': 'J18.9', 'CPT': '71046'}, 'final_report': 'Final Report: Pneumonia for patient Ali Ahmadi. Confidence: 1.00.', 'memory_status': 'Consolidation Successful'}
Processing: sample_images/011.jpeg
-> artifacts returned: {'user_request': 'sample_images/011.jpeg', 'patient_data': {'patient_id': 'sample_images/011.jpeg', 'name': 'Ali Ahmadi', 'age': 45}, 'analysis_findings': {'pathology': 'Pneumonia', 'confidence': 0.9256}, 'coding_result': {'ICD_10': 'J18.9', 'CPT': '71046'}, 'final_report': 'Final Report: Pneumonia for patient Ali Ahmadi. Confidence: 0.93.', 'memory_status': 'Consolidatio

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# Download submission CSV locally
# ----------------------------------------
from google.colab import files
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>